# GFD-Poisson
Generalized Finite Differences Methods for numerically solve Poisson Equation on highly irregular regions.

Below are examples of the implementation of a Generalized Finite Difference Method to compute an approximation to the solution of the Poisson equation in 2D.

The problem to solve is
\begin{equation}
     \frac{\partial^2}{\partial x^2} \phi(x,y) + \frac{\partial^2}{\partial y^2} \phi(x,y) = -f(x,y)
\end{equation}

## Logically Rectangular Meshes

Due to advances in the numerical generation of structured meshes, it is possible to mesh highly irregular regions.

For example, the CUI region (standing for Cuitzeo Lake),

<center><img src="/Notebook_Figures/fig01.png" alt="Cuitzeo Lake region" width="240" height="240"/></center>

can be meshed as

<center><img src="/Notebook_Figures/fig02.png" alt="Cuitzeo mesh" width="240" height="240"/></center>

One of the greatest advantages of structured grides is that it is possible to take advantage of the data structure to make numerical approximations over them.

### Python Modules Importation
The first thing that we have to do is to import all the Python modules that will be used for the approximation. In the following code, we import the Modules:
- scipy.io.loadmat: In order to import data from Matlab files.
- numpy: In order to be able to use numerical computing.
- matplotlib.pyplot: In order to be able to graph the results.
- matplotlib.cm: To use different colormaps for the graphics.

In [11]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

### Getting the region data
Now, it is required to have all the data of the region loaded into Python variables.

In [12]:
mat  = loadmat('Data/Meshes/CUI21.mat')                                         # The data is loaded form a .mat file.
x  = mat['x']                                                                   # The x-coordinates are saved in x.
y  = mat['y']                                                                   # The y-coordinates are saved in y.

### Variable Initialization
In order to correctly compute the approximation, some variables are required to be initialized.

In [13]:
me    = x.shape                                                                 # The size of the mesh is found.
m     = me[0]                                                                   # The number of nodes in x.
n     = me[1]                                                                   # The number of nodes in y.
err   = 1                                                                       # err initialization in 1.
iter  = 0                                                                       # Number of iterations.
tol   = 1e-16                                                                   # The tolerance is defined.
m_it  = 400                                                                     # Maximum number of iterations.
u_ap  = np.zeros([m,n])                                                         # u_ap initialization with zeros.
u_ex  = np.zeros([m,n])                                                         # u_ex initialization with zeros.
Gamma = np.zeros([m,n,9])                                                       # Gamma initialization with zeros.

### Boundary conditions
The boundary conditions must be defined. For this example, we will define the condition as:

\begin{equation}
    \phi = 2e^{2x+y}
\end{equation}

In [14]:
def phi(x,y):                                                                   # Definition of the Phi function.
    fun = 2*np.exp(2*x+y)                                                       # Phi condition.
    return fun                                                                  # Return phi.

for i in range(m):                                                              # For each of the nodes on the x boundaries.
    u_ap[i, 0]   = phi(x[i, 0],   y[i, 0])                                      # The boundary condition is assigned at the first y.
    u_ap[i, n-1] = phi(x[i, n-1], y[i, n-1])                                    # The boundary condition is assigned at the last y.
for j in range(n):                                                              # For each of the nodes on the y boundaries.
    u_ap[0,   j] = phi(x[0,   j], y[0,   j])                                    # The boundary condition is assigned at the first x.
    u_ap[m-1, j] = phi(x[m-1, j], y[m-1, j])                                    # The boundary condition is assigned at the last x.

### Gammas
The Gammas are the "weights" that will be used in order to have a a Finite Difference-based approximation.

In [15]:
L = np.vstack([[0], [0], [2], [0], [2]])                                        # The values of the differential operator are assigned.

for i in range(1,m-1):                                                          # For each of the nodes in x.
    for j in range(1,n-1):                                                      # For each of the nodes in y.
        dx = np.array([x[i + 1, j]   - x[i, j], x[i + 1, j + 1] - x[i, j], \
                       x[i, j + 1]   - x[i, j], x[i - 1, j + 1] - x[i, j], \
                       x[i - 1, j]   - x[i, j], x[i - 1, j - 1] - x[i, j], \
                       x[i, j - 1]   - x[i, j], x[i + 1, j - 1] - x[i, j]])     # dx is computed.
            
        dy = np.array([y[i + 1, j]   - y[i, j], y[i + 1, j + 1] - y[i, j], \
                       y[i, j + 1]   - y[i, j], y[i - 1, j + 1] - y[i, j], \
                       y[i - 1, j]   - y[i, j], y[i - 1, j - 1] - y[i, j], \
                       y[i, j - 1]   - y[i, j], y[i + 1, j - 1] - y[i, j]])     # dy is computed.
            
        M = np.vstack([[dx], [dy], [dx**2], [dx*dy], [dy**2]])                  # M matrix is assembled.
        M = np.linalg.pinv(M)                                                   # The pseudoinverse of matrix M.
        YY = M@L                                                                # M*L computation.
        Gem = np.vstack([-sum(YY), YY])                                         # Gamma values are found.
        for k in range(9):                                                      # For each of the Gamma values.
            Gamma[i,j,k] = Gem[k]  

### Generalized Finite Differences Method
With the computed Gammas, it is possible to apply a Finite Difference Method in order to get an approximation.

It is required to have the "right-hand-size" of Poisson equation, for this case, this will be defined as:
\begin{equation}
    f = 10e^{2x+y}
\end{equation}

In [16]:
def f(x,y):                                                                     # Definition of the f function.
    fun = 10*np.exp(2*x+y)                                                      # f condition.
    return fun                                                                  # Return f.

while err >= tol and iter <= m_it:                                              # Check for iterations and tolerance.
    err = 0                                                                     # Error becomes zero to be able to update.
    for i in range(1,m-1):                                                      # For each of the nodes on the x axis.
        for j in range(1,n-1):                                                  # For each of the nodes on the y axis.
            t = (f(x[i, j], y[i, j]) - (            \
                Gamma[i, j, 1]*u_ap[i + 1, j    ] + \
                Gamma[i, j, 2]*u_ap[i + 1, j + 1] + \
                Gamma[i, j, 3]*u_ap[i    , j + 1] + \
                Gamma[i, j, 4]*u_ap[i - 1, j + 1] + \
                Gamma[i, j, 5]*u_ap[i - 1, j    ] + \
                Gamma[i, j, 6]*u_ap[i - 1, j - 1] + \
                Gamma[i, j, 7]*u_ap[i    , j - 1] + \
                Gamma[i, j, 8]*u_ap[i + 1, j - 1]))/Gamma[i, j, 0]              # u_ap is calculated at the central node.
            err = max(err, abs(t - u_ap[i, j]));                                # Error computation.
            u_ap[i,j] = t;                                                      # The previously computed value is assigned.
    iter += 1 

###

### Theoretical Solution
For this example, the theoretical solution can be computed as the boundary condition, and it will be computed in order to have a point of comparison for the computed numerical solution.

In [17]:
for i in range(m):                                                              # For all the nodes on x.
    for j in range(n):                                                          # For all the nodes on y.
        u_ex[i,j] = phi(x[i,j], y[i,j])                                         # The theoretical solution is computed.

### Error
The error can be computed as the mean square error given by
\begin{equation}
    \mid e \mid^2 = \left(\sqrt{\sum_{i,j} (u_{i,j} - U_{i,j})}\right) A_{i,j}
\end{equation}

where $u_{i,j},j$ and $U_{i,j}$ are the computed and theoretical solution, respectively, and $A_{i,j}$ is the area of the corresponding polygon given by the neighbor nodes.

In [18]:
def PolyArea(x,y):
    area = 0.5*np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))      # Area of the polygon given by the x and y coordinates.
    return area

In [20]:
err   = 0                                                                       # err initialization with 0.
area = np.zeros([m,n])                                                          # area initialization with zeros.

for i in np.arange(1,m-1):                                                      # For each of the nodes on the x axis.
    for j in np.arange(1,n-1):                                                  # For each of the nodes on the y axis.
        px = np.array([x[i+1, j], x[i+1, j+1], x[i, j+1], x[i-1, j+1], \
                       x[i-1, j], x[i-1, j-1], x[i, j-1], x[i+1, j-1]])         # The x-values of the polygon are stored.
        py = np.array([y[i+1, j], y[i+1, j+1], y[i, j+1], y[i-1, j+1], \
                       y[i-1, j], y[i-1, j-1], y[i, j-1], y[i+1, j-1]])         # The y-values of the polygon are stored.
        area[i,j] = PolyArea(px,py)                                             # Area computation.

for i in np.arange(m):                                                          # For each of the nodes on the x axis.
    for j in np.arange(n):                                                      # For each of the nodes on the y axis.
        err = err + area[i,j]*(u_ap[i,j] - u_ex[i,j])**2                        # Mean error computation.
    
err = np.sqrt(err)                                                              # The mean square error.

print('The mean square error in the mesh is: ', err)

The mean square error in the mesh is:  0.002723586345478777


### Graphics
It is possible to graph the computed solution along with the theoretical solution to get a graphical idea on how good the approximation is.

In [ ]:
min  = u_ex.min()
max  = u_ex.max()
    
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(8, 4))
    
ax1.set_title('Approximation')
ax1.set_zlim([min, max])
ax1.plot_surface(x, y, u_ap, cmap=cm.coolwarm)
    
ax2.set_title('Theoretical Solution')
ax2.set_zlim([min, max])
ax2.plot_surface(x, y, u_ex, cmap=cm.coolwarm)

plt.show()